In [ ]:
import logging
import os
import pandas as pd

from pathlib import Path

%load_ext autoreload
%autoreload 2

# Disable logging of the program in the notebook
os.environ["LOGLEVEL"] = "CRITICAL"

logging.basicConfig()
logging.root.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)

logger = logging.getLogger('__main__')
logger.setLevel(os.environ.get("LOGLEVEL", logging.FATAL))

pybooklogger = logging.getLogger('pybook')
pybooklogger.setLevel(logging.DEBUG)

import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.path.abspath("__file__")), '..')))
from setup import *
from tad_help import TadRunner

# Track Layout
Calculate the layout of the dutch railway system


In [ ]:
layout_file =   "../data/prorail/parsed/netherlands-schiphol.json"
start = time.time()
layout = Layout(layout_file)
end = time.time()
print(f"Computed the routing graph for entire Dutch railway network in {end - start} seconds.")
print(f"Number of track parts in the track graph: {len(layout.g.nodes)} with {len(layout.g.edges)} connections.")
print(f"Number of blocks in the routing graph: {len(layout.g_block.nodes)} with {len(layout.g_block.edges)} routes.")

In [ ]:
edges_df = pd.DataFrame({"Outgoing routes": [len(n.outgoing) for n in layout.g_block.nodes.values() if len(n.outgoing) <= 25]})
hist = edges_df.hist(bins=25, )
plt.xlabel("Number of outgoing routes")
plt.ylabel("Number of occurrences")
plt.show()

# Experiment Runtime
Take a route of an agent with many stops, and run from start to every stop as an experiment

## Scenario

In [ ]:
scenario_files = Path("../data/prorail/scenarios/SHL")
save_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), "results", "case_study_eurostar")
Path(save_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
os.environ["LOGLEVEL"] = "INFO"
path_data = []
time_data = []
comp_data = []

for scenario_file in os.listdir(scenario_files):
    tad_exp = TadRunner(layout, scenario_files / scenario_file, save_dir)
    experiments = tad_exp.run("1800", "e", "RTD", "SHL", default_direction=1, timeout=1200, 
                              startTime=0, trainTypes=["EUROSTAR"], stop=[])
    experiments[0].metadata = {'color': 'Red',   'label': '@MAEDeR',  'offset': 0, 'linestyle': 3}
    experiments[1].metadata = {'color': 'Blue',  'label': 'FlexSIPP', 'offset': 0, 'linestyle': 3}
    experiments[2].metadata = {'color': 'Green', 'label': 'rSIPP',    'offset': 0, 'linestyle' :3}

    tad_exp.plot(experiments, save="eurostar", y_offset=2100, y_range=1200, include_expected_arrival=False)
    tad_exp.plot([experiments[0]], save="eurostar/atsipp", y_offset=3040, include_expected_arrival=False)
    tad_exp.plot([experiments[1]], save="eurostar/flexsipp", y_offset=1240, include_expected_arrival=False)
    tad_exp.plot([experiments[2]], save="eurostar/rsipp", y_offset=0, include_expected_arrival=False)

    path_data.extend(get_path_data(experiments, tad_exp.agent_df,scenario=scenario_file))
    time_data.extend([exp.get_running_time() | exp.get_label() | {'scenario': scenario_file} for exp in experiments])
    comp_data.extend([exp.get_complexity()   | exp.get_label() | {'scenario': scenario_file} for exp in experiments])

path_df = pd.DataFrame(path_data)
path_df.to_csv(save_dir + "/eurostar.csv")

time_df = pd.DataFrame(time_data)
time_df.to_csv(save_dir + "/eurostar-time.csv")

comp_df = pd.DataFrame(comp_data)
comp_df.to_csv(save_dir + "/eurostar-comp.csv")


In [ ]:
experiments[0].metadata = {'color': 'Red',   'label': '@MAEDeR',    'offset': 0, 'linestyle': 3}
experiments[1].metadata = {'color': 'Blue',  'label': 'FlexSIPP', 'offset': 0, 'linestyle': 3}
experiments[2].metadata = {'color': 'Green', 'label': 'rSIPP', 'offset': 0, 'linestyle' :3}

tad_exp.plot(experiments, save="eurostar", y_offset=2100, y_range=1200, include_expected_arrival=False)
tad_exp.plot([experiments[0]], save="eurostar/atsipp", y_offset=3040, include_expected_arrival=False)
tad_exp.plot([experiments[1]], save="eurostar/flexsipp", y_offset=1240, include_expected_arrival=False)
tad_exp.plot([experiments[2]], save="eurostar/rsipp", y_offset=1240, include_expected_arrival=False)

path_data = get_path_data(experiments, tad_exp.agent_df, scenario=scenario_file)
time_data = [exp.get_running_time() | exp.get_label() | {'scenario': scenario_file} for exp in experiments]
comp_data = [exp.get_complexity()   | exp.get_label() | {'scenario': scenario_file} for exp in experiments]

path_df = pd.DataFrame(path_data)
path_df.to_csv(save_dir + "/eurostar.csv")

time_df = pd.DataFrame(time_data)
time_df.to_csv(save_dir + "/eurostar-time.csv")

comp_df = pd.DataFrame(comp_data)
comp_df.to_csv(save_dir + "/eurostar-comp.csv")

In [ ]:
path_df = pd.read_csv(save_dir + "/eurostar.csv", index_col=0)
path_df

In [ ]:
import re

allowed_delay = 180

def td_str(td):
    return ':'.join(re.split(r'[:.]+', str(td)) [1:3])

def extract_tipping_point(df):
    def apply_func(df):

        result=df.groupby("Delay Location").agg({
            "alpha": "min",
            "beta": "max",
            "Delay Amount": "max",
        })
        result = result.loc[result['alpha'] < 900]
        result["Tipping Point"] = result["alpha"].apply(lambda x: td_str(timedelta(seconds=x)))

        def tp_finder(x):
            new_tp = x['beta'] - max(0, x['Delay Amount'] - allowed_delay)
            if new_tp > 0:
                return td_str(timedelta(seconds=new_tp))
            return "-"

        # result[f"Tipping Point ({allowed_delay}s)"] = result.apply(tp_finder, axis=1)
        return result.sort_values("Tipping Point", ascending=True).drop(columns=["alpha", "beta", "Delay Amount"])

    df["Delay Location"] = df["Delay Location"].str.split("|").apply(lambda x: x[0])
    df = df.groupby(by='Train').apply(apply_func, include_groups=False)
    return df

tp_df = path_df.rename(columns={
    "delay_amount": "Delay Amount",
    "delay_location": "Delay Location",
    "trainNumber": "Train",
    "scenario": "Scenario",
    "label": "Label",
})

tp_df["Scenario"] = tp_df["Scenario"].apply(lambda x: x.split(".")[0])
tp_df["Delay Location"] = tp_df["Delay Location"].apply(lambda x: x.split("-")[1])
tp_df = tp_df.groupby(["Scenario"]).apply(extract_tipping_point, include_groups=False)
# tp_df = tp_df.loc[tp_df.index.get_level_values('Train').astype(str).str.startswith("43")]
# tp_df = tp_df.loc[tp_df.index.get_level_values('Delay Location').astype(str).str.startswith("Ledn")]
tp_df.to_latex(save_dir + "/eurostar.tex", escape=True)
tp_df

### Blocking staircase diagram
Showing the route of the agent with the most stops, its quite long.


In [ ]:
experiments[0].metadata = experiments[0].metadata | {'label': ''}
experiments[1].metadata = experiments[0].metadata | {'label': '_buffer'}
for exp in experiments:
    exp.s.plot(exp.agent.id, exp.block_intervals, exp.buffer_times, exp.recovery_times, False, start="RTD", end="GV", savefig=f"{save_dir}/blocking_staircase{exp.metadata['label'].replace(' ', '_')}.png")

In [ ]:
for exp in experiments:
    exp.agent.start_time = 1503

In [ ]:
timeout = 600
run_experiments([experiments[1]], timeout)

## Results

### ATF Plot

In [ ]:
experiments[0].metadata = {'color': 'Red', 'label': 'No flexibility', 'offset': 0,  'linestyle': 3}
experiments[1].metadata = {'color': 'Green', 'label': 'Buffer time', 'offset': 0,   'linestyle': 3}
experiments[2].metadata = {'color': 'Blue', 'label': 'Recovery time', 'offset': -0, 'linestyle': 3}

kwargs = {"min_x": 1503, "max_x": 2100, "min_y": 50 * 60, "max_y": 80 * 60}
plot_experiments(experiments, **kwargs)


experiments[0].metadata = {'color': 'Red', 'label': 'No flexibility', 'offset': 0, 'linestyle': 3}
experiments[1].metadata = {'color': 'Green', 'label': 'Buffer time', 'offset': 0,  'linestyle': 3}
experiments[2].metadata = {'color': 'Blue', 'label': 'Recovery time', 'offset': 0, 'linestyle': 3}

plot_experiments([experiments[0]], **kwargs)
plot_experiments([experiments[1]], **kwargs)
plot_experiments([experiments[2]], **kwargs)

### Time statistics

In [ ]:
def sum_cols(df1, cols, name):
    df2 = df1.drop(columns=cols)
    df2[name] = df1[cols].sum(axis=1)
    return df2

time_df = pd.DataFrame([exp.get_running_time() for exp in experiments], index=[exp.metadata['label'] for exp in experiments])

setup_cols = ["track graph creation", "routing graph creation"]
recompute_cols = ["unsafe interval generation", "safe interval generation", "bt and crt generation", "converting routes to blocks"]
search_cols = ["FlexSIPP search time"]

time_df = sum_cols(time_df, setup_cols, "Setup Time")
time_df = sum_cols(time_df, recompute_cols, "Recompute Time")
time_df = sum_cols(time_df, search_cols, "Search Time")
time_df

### Search Node Statistics


In [ ]:
nodes_df = pd.DataFrame([exp.get_complexity() for exp in experiments], index=[exp.metadata['label'] for exp in experiments])

nodes_df

### Output paths found


In [ ]:
for key, value in experiments[2].results[3].items():
    delayed_trains = {i: v for i,v in enumerate(value[0][4]) if float(v[0]) > 0}
    print(f"{key.replace('r-', '')}\nearliest departure: {int(min(float(value[0][1]), float(value[0][2])) / 60)}\ndepart before: {int(float(value[0][2]) / 60)}\narrive at: {int((float(value[0][1]) + float(value[0][3])) / 60)}\ndelays trains: {delayed_trains}")

### Path statistics


In [ ]:
for exp in experiments:
    print(f"Different paths found for {exp.metadata['label']}: {sum(exp.results[2].values())}")